# Testing

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

from getScreenTime import get_screen_time_data
from getAppName import get_app_name

45


## get data

In [25]:
#gets (appname, (year, month, hour, minute, second, dayOfWeek))
data = get_screen_time_data(758000000, 758935000) # jan 7 10pm - January 18, January 18, 2025 6:16:40 PM

# appname to index
apps = get_app_name(758000000, 758935000)

In [41]:

len_data = len(data)
len_apps = len(apps)

X_test = np.zeros((len_data, 4))
Y_test = np.zeros((len_data,))

for i in range(len_data):
    x = data[i][1]
    y = data[i][0]

    X_test[i][0] = x[1]#/12
    X_test[i][1] = x[2]#/24
    X_test[i][2] = x[3]#/60
    X_test[i][3] = x[5]#/6

    if y not in apps:
        Y_test[i] = 1
    else:
        ind = apps.index(y)
        Y_test[i] = ind/len_apps
print(X_test)

[[ 1. 22. 33.  2.]
 [ 1. 23.  1.  2.]
 [ 1. 23. 30.  2.]
 ...
 [ 1. 16. 46.  6.]
 [ 1. 17. 20.  6.]
 [ 1. 17. 52.  6.]]


## model

In [42]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(100, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_test, Y_test, epochs=10)

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.4890 - loss: 2.9978    
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - accuracy: 0.7512 - loss: 0.8172
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.6983 - loss: 0.6836
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - accuracy: 0.7152 - loss: 0.6230
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step - accuracy: 0.7511 - loss: 0.6161
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 979us/step - accuracy: 0.7760 - loss: 0.5895
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.7447 - loss: 0.5572
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.7683 - loss: 0.5832
Epoch 9/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.7199 - loss: 0.5332
Epoch 10/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7303 - loss: 0.5585
Epoch 11/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.7514 - loss: 0.5693
Epoch 12/100
17/17 ━━━━━━━━━━━━━━━━━━━━